In [62]:
%matplotlib inline

import pandas as pd
import numpy as np

from pandas.api.types import CategoricalDtype

from collections import defaultdict, Counter

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import datetime as dt
import matplotlib.dates as mdates

import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf

import dataStatsAnalysis as dsa
import dataStatsPlotting as dsp

dsp.SetParams()

In [63]:
class UnimplementedMethodException(Exception):
    """Exception if someone calls a method that should be overridden."""


class HypothesisTest():
    """Hypothesis test superclass. 

    This class cannot be used as is. 
    It is to be used to construct hypothesis tests 
    for various different test statistics. 
    See the existing child classes below for examples.
    """
    def __init__(self, data, tail='right', iters=1000):
        self.data = data
        self.tail = tail
        self.iters = iters
        self.PrepareData(data)
        self.TestStat()
        self.sampling_dist, self.rv = self.ComputeRv() # pylint: disable=assignment-from-no-return

    # Provide the functionality to convert the data into the format needed 
    # for use in ComputeRv and Power functions. 
    # Ex. Convert to array, split data into component groups, etc. 
    # The self data variables must be created in the function, not returned. 
    # See child classes for examples
    def PrepareData(self, data):
        UnimplementedMethodException()
        
    # This function only needs to be written in the case of a null hypothesis based test. 
    # The self.test_stat needs to be created in the function, not returned. 
    # In the case of an alternative hypothesis based test 
    # test_stat will be provided via a class parameter. 
    # See child classes for examples
    def TestStat(self):
        pass
    
    # Provide the functionality that computes the sampling distribution and rv for the data.
    # Both the sampling distribution and the rv need to be returned by the function 
    # See child classes for examples
    def ComputeRv(self):
        UnimplementedMethodException()
        
    # Provide the functionality that computes the power by running multiple iterations of the hypothesis test.
    # The code in the for loop must first create new data for the run, 
    # which simulates taking another sample from the population, and then run the hypothesis test.
    # See child classes for examples
    def Power(self):
        UnimplementedMethodException()
    
    def PValue(self):
        """Computes the p-value for the hypothesis test.

        returns: float p-value
        """
        if self.tail == 'left':
            pvalue = self.rv.cdf(self.test_stat) # pylint: disable=no-member
        elif self.tail == 'right':
            pvalue = 1 - self.rv.cdf(self.test_stat) # pylint: disable=no-member
        else:
            raise Exception('The value of \'tail\' can only be either \'left\' or \'right\'')

        return pvalue

    def MinMaxTestStat(self):
        """Returns the smallest and largest test statistics in the sampling distribution.
        """
        return min(self.sampling_dist), max(self.sampling_dist)

    def PlotCdf(self):
        """Draws a Cdf with a vertical line at the test stat.
        """      
        plt.plot(self.rv.xk, self.rv.cdf(self.rv.xk), color='C0', lw=2) # pylint: disable=no-member
        
        plt.axvline(self.test_stat, color='C1', lw=1.3) # pylint: disable=no-member

In [64]:
class HTOnewayAnova(HypothesisTest):
    """A one-way ANOVA hypothesis test. 
    Uses permutation of the supplied data sequences to build the null hypothesis 
    and f-statistic sampling distribution. 
    Accepts data in the form of a list of data sequences (group_1, group_2... group_n).

    Parameters
    ----------
    data (array-like):
        A list or tuple of data sequences (group_1, group_2... group_n)
    tail (str):
        The tail of the distribution to be used in the PValue function
        Accepts only 'right' or 'left'
        Defaults to 'right'
    iters (int):
        The number of iterations to run in the ComputeRv function 
        Defaults to 1000
    
    Attributes
    ----------
    data:
        The original data
    test_stat:
        The test statistic used in the hypothesis test
    sampling_dist:
        The sampling distribution generated by resampling
    rv:
        A scipy.stats discrete_rv object (random variable) 
        that represents the sampling distribution
        This object provides numerous useful attributes and methods
        See the discrete_rv documentation for details

    Methods
    -------
    PValue():
        Computes the p-value for the hypothesis test
    Power(alpha=0.05, num_runs=1000):
        Computes the power of the hypothesis test
        alpha: the significance level for the hypothesis test, default=0.05
        num_runs: the number of hypothesis tests to run, default=1000
    MinMaxTestStat():
        Returns the smallest and largest test statistics in the sampling distribution
    PlotCdf():
        Draws a Cdf of the distribution with a vertical line at the test stat
    """
    def PrepareData(self, data):
        self.pooled_data = np.hstack(data)
        
    def TestStat(self):
        self.test_stat, _ = stats.f_oneway(*self.data)
        
    def ComputeRv(self):    
        # Initialize the sampling distribution list 
        # that will hold the f-stats calculated for each iteration 
        f_stats = []
        
        for _ in range(self.iters):
            # Permute the pooled data
            pooled_data_perm = np.random.permutation(self.pooled_data)
            
            # For each item in the orignal data 
            # pull out the correct number of permuted values 
            # and store the new sequences in a list
            data_perm_list = []
            for x in self.data:
                x_perm = pooled_data_perm[:len(x)]
                data_perm_list.append(x_perm)
    
                pooled_data_perm = pooled_data_perm[len(x):]
                
            # Compute the f-stat for the permuted data 
            # and add it to the sampling distribution list
            f_stat, _ = stats.f_oneway(*data_perm_list)
            f_stats.append(f_stat)
            
        return np.array(f_stats), dsa.DiscreteRv(f_stats)
    
    def Power(self, alpha=0.05, num_runs=1000):
        """Computes the power of the hypothesis test. 

        Args
        ----
        alpha (float):
            The significance level for the hypothesis test.
            Must be between 0 and 1. Defaults to 0.05
        num_runs (int):
            The number of times to run the hypothesis test to compute power.
            Defaults to 1000.

        Returns
        -------
        power:
            Computed as the percentage of significant pvalues in num_runs of the test.
            Returned value is between 0 and 1.
        """    
        pvalue_count = 0
        
        for _ in range(num_runs):
            # Simulate resampling from the population
            run_data_resample_list = []
            for x in self.data:
                x_run_resample = np.random.choice(x, size=len(x), replace=True)
                run_data_resample_list.append(x_run_resample)
            
            # Run the hypothesis test with run_data
            test = HTOnewayAnova(run_data_resample_list, tail=self.tail, iters=100)
            pvalue = test.PValue()
            
            if pvalue < alpha:
                pvalue_count += 1
            
        return pvalue_count / num_runs

In [65]:
# Data to use: comes from the scipy.stats.f_oneway example
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.f_oneway.html

tillamook = [0.0571, 0.0813, 0.0831, 0.0976, 0.0817, 0.0859, 0.0735,
             0.0659, 0.0923, 0.0836]
newport = [0.0873, 0.0662, 0.0672, 0.0819, 0.0749, 0.0649, 0.0835,
           0.0725]
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764,
           0.0689]
tvarminne = [0.0703, 0.1026, 0.0956, 0.0973, 0.1039, 0.1045]

In [66]:
data = [tillamook, newport, petersburg, magadan, tvarminne]

In [67]:
np.mean(tillamook), np.mean(newport), np.mean(petersburg), np.mean(magadan), np.mean(tvarminne)

(0.0802,
 0.07479999999999999,
 0.10344285714285714,
 0.0780125,
 0.09570000000000001)

In [68]:
stats.f_oneway(*data)

F_onewayResult(statistic=7.121019471642447, pvalue=0.0002812242314534544)

In [69]:
# # Try permutation , petersburg_perm, magadan_perm, tvarminne_perm

# data = [tillamook, newport, petersburg, magadan, tvarminne]
# pooled_data = np.hstack(data)
# pooled_data_perm = np.random.permutation(pooled_data)

# data_perm_list = []
# for x in data:
#     x_perm = pooled_data_perm[:len(x)]
#     data_perm_list.append(x_perm)
    
#     pooled_data_perm = pooled_data_perm[len(x):]

# data_perm_list, stats.f_oneway(*data_perm_list)
# # pooled_data_perm = np.random.permutation(pooled_data)
# # tillamook_perm = pooled_data_perm[:len(tillamook)]
# # newport_perm = pooled_data_perm[len(tillamook):len(tillamook)+len(newport)]
# # petersburg

# # tillamook_perm, newport_perm

In [70]:
data

[[0.0571,
  0.0813,
  0.0831,
  0.0976,
  0.0817,
  0.0859,
  0.0735,
  0.0659,
  0.0923,
  0.0836],
 [0.0873, 0.0662, 0.0672, 0.0819, 0.0749, 0.0649, 0.0835, 0.0725],
 [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105],
 [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764, 0.0689],
 [0.0703, 0.1026, 0.0956, 0.0973, 0.1039, 0.1045]]

In [71]:
# Might be working now, but hard to tell because the pvalues are so low
# Should try changing the numbers a bit to get a higher pvalue
# Still need to create the Power part
htanova = HTOnewayAnova(data)

In [72]:
htanova.PValue()

0.0009999999999992237

In [73]:
htanova.MinMaxTestStat()

(0.0165219507655746, 8.48187552612493)

In [74]:
htanova.test_stat

7.121019471642447

In [75]:
htanova.sampling_dist

array([0.96012691, 0.95204698, 0.31561422, 0.64679999, 1.1771413 ,
       0.84316664, 1.54682619, 2.0433977 , 1.23143929, 0.18277863,
       0.60509812, 0.52596846, 0.46851248, 0.4241688 , 0.45681736,
       0.95548683, 1.64163648, 0.95642035, 0.2687747 , 5.19579804,
       0.51379516, 0.55535965, 0.70248215, 1.00692274, 1.51779378,
       0.48867361, 0.41735847, 2.77694889, 0.23100218, 0.61298012,
       0.87044732, 2.61683076, 0.61907443, 1.85308621, 1.56666591,
       0.71320869, 0.88433639, 1.50245968, 1.53796146, 0.95051549,
       0.08783491, 0.82438445, 1.13894323, 0.90226289, 1.26040532,
       2.65412237, 1.14333052, 1.71253322, 0.28722253, 0.19905155,
       0.89048246, 0.43911821, 2.26396908, 0.83900452, 0.59812311,
       2.18112943, 1.58839728, 4.10290901, 0.67244337, 0.41575349,
       1.08192488, 0.50094181, 0.16316576, 0.5236236 , 1.46661943,
       1.54546254, 0.56310171, 1.09809799, 0.84702794, 0.40602349,
       1.255212  , 0.30461877, 0.58748964, 1.44370728, 3.13613

In [76]:
# I think it is working. The power is 1.0 because the p-value is so small. The test pretty much always gives significant result
htanova.Power(num_runs=100)

0.99

In [77]:
htanova.test_stat

7.121019471642447

In [78]:
tillamook = [0.0571, 0.0813, 0.0831, 0.0976, 0.0817, 0.0859, 0.0735,
             0.0659, 0.0923, 0.0836]
newport = [0.0873, 0.0662, 0.0672, 0.0819, 0.0749, 0.0649, 0.0835,
           0.0725]
new_petersburg = [0.0777, 0.1, 0.0817, 0.0807, 0.0887, 0.1, 0.0815]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764,
           0.0689]
new_tvarminne = [0.0803, 0.1, 0.0805, 0.0973, 0.0837, 0.1]

In [79]:
new_data = [tillamook, newport, new_petersburg, magadan, new_tvarminne]

In [80]:
# To use G-power, calculate the means of each data sequence
np.mean(tillamook), np.mean(newport), np.mean(new_petersburg), np.mean(magadan), np.mean(new_tvarminne)

(0.0802, 0.07479999999999999, 0.0871857142857143, 0.0780125, 0.0903)

In [81]:
# To use G-Power, calculate the average standard deviation
(np.std(tillamook) + np.std(newport) + np.std(new_petersburg) + np.std(magadan) + np.std(new_tvarminne))/5

0.009815152852906583

In [82]:
htanova2 = HTOnewayAnova(new_data)

In [83]:
htanova2.test_stat

2.486048405682797

In [84]:
htanova2.PValue()

0.06599999999999928

In [85]:
# Seems to be working, GPower gives a result of 0.74
htanova2.Power(num_runs=100)

0.77

In [86]:
stats.f_oneway(*new_data)

F_onewayResult(statistic=2.486048405682797, pvalue=0.061842209191269946)

In [87]:
htanova2.test_stat

2.486048405682797

#### Post hoc analysis

In [88]:
# Can I just use my difference in means hypothesis tests to find significant differences between pairs of data sequences?

In [113]:
from itertools import combinations
from math import factorial

In [182]:
def AnovaPostHoc(data, alpha=0.05):
    num_comparisons = int(factorial(len(data))/(2*factorial(len(data)-2)))
    corrected_alpha = 0.05/num_comparisons
    print(corrected_alpha)
    enum_data = enumerate(data)
    
    for pair in combinations(enum_data, 2):
        test = dsa.HTDiffMeansH0((pair[0][1], pair[1][1]))
        pvalue = test.PValue()
        significant = 'Y' if pvalue < corrected_alpha else 'N' 
        print(pair[0][0], pair[1][0], ':', '{:.3f}'.format(pvalue), significant)


In [183]:
AnovaPostHoc(new_data)

0.005
0 1 : 0.303 N
0 2 : 0.238 N
0 3 : 0.702 N
0 4 : 0.102 N
1 2 : 0.021 N
1 3 : 0.601 N
1 4 : 0.018 N
2 3 : 0.136 N
2 4 : 0.592 N
3 4 : 0.074 N


In [186]:
AnovaPostHoc(data)

0.005
0 1 : 0.273 N
0 2 : 0.001 Y
0 3 : 0.703 N
0 4 : 0.032 N
1 2 : 0.000 Y
1 3 : 0.559 N
1 4 : 0.006 N
2 3 : 0.002 Y
2 4 : 0.383 N
3 4 : 0.033 N


In [169]:
# Compare this with statsmodels tukeyhsd
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [167]:
# Setting this up is crazy complicated
# Found a way to do it here:
# https://python.hotexamples.com/examples/statsmodels.stats.multicomp/-/pairwise_tukeyhsd/python-pairwise_tukeyhsd-function-examples.html
x = []
label = []

for grp_num in range(len(data)):
    x.extend(data[grp_num])
    label.extend([grp_num]*len(data[grp_num]))

results = pairwise_tukeyhsd(np.array(x), np.array(label))
print(results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     0      1  -0.0054 0.8898 -0.0226  0.0118  False
     0      2   0.0232 0.0057  0.0054  0.0411   True
     0      3  -0.0022    0.9 -0.0194   0.015  False
     0      4   0.0155 0.1446 -0.0032  0.0342  False
     1      2   0.0286  0.001  0.0099  0.0474   True
     1      3   0.0032    0.9 -0.0149  0.0213  False
     1      4   0.0209 0.0318  0.0013  0.0405   True
     2      3  -0.0254 0.0037 -0.0442 -0.0067   True
     2      4  -0.0077 0.7791 -0.0279  0.0124  False
     3      4   0.0177 0.0928 -0.0019  0.0373  False
----------------------------------------------------


In [161]:
# This is the full function from which I got the method for using pairwise_tukey above
def test_ANOVA_and_T_HSD(array_of_arrays):
    ANOVA_f, ANOVA_p = stats.f_oneway(*array_of_arrays)

    if ANOVA_p < 0.05:
        x = []
        label = []
        for grp_num in range(len(array_of_arrays)):
            x.extend(array_of_arrays[grp_num])
            label.extend([grp_num]*len(array_of_arrays[grp_num]))
        
        tukey_HSD_result = pairwise_tukeyhsd(np.array(x), np.array(label))
        
        return ANOVA_p, tukey_HSD_result
    else:
        return ANOVA_p, False

In [162]:
test_ANOVA_and_T_HSD(data)

[0.0571, 0.0813, 0.0831, 0.0976, 0.0817, 0.0859, 0.0735, 0.0659, 0.0923, 0.0836, 0.0873, 0.0662, 0.0672, 0.0819, 0.0749, 0.0649, 0.0835, 0.0725, 0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105, 0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764, 0.0689, 0.0703, 0.1026, 0.0956, 0.0973, 0.1039, 0.1045]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4]


(0.0002812242314534544,
 <statsmodels.sandbox.stats.multicomp.TukeyHSDResults at 0x18bee9c9390>)